In [169]:
import boto3
from file_extraction import *
from to_dataframe import *
import pandas as pd
import datetime

In [170]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}

In [171]:
extract_file_type(s3_client, 'Talent', files_dict, 'json')
applicants = files_dict['json']

df = convert_all_to_df(applicants, 'json')

df

,name,date,tech_self_score,strengths,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest
0,Stillmann Castano,22/08/2019,"{'C#': 6, 'Java': 5, 'R': 2, 'JavaScript': 2}",[Charisma],"[Distracted, Impulsive, Introverted]",Yes,Yes,Yes,Pass,Business
0,Hilary Willmore,01/08/2019,"{'Python': 1, 'C#': 4, 'Java': 2, 'C++': 4}","[Patient, Curious, Problem Solving]","[Overbearing, Chatty, Indifferent]",No,Yes,Yes,Fail,Data
0,Efrem Whipple,22/08/2019,"{'Ruby': 4, 'C++': 4}","[Courteous, Independent, Patient]","[Introverted, Impulsive, Anxious]",Yes,Yes,Yes,Pass,Business
0,Sydel Fenne,28/08/2019,"{'Java': 3, 'SPSS': 4}",[Passionate],"[Perfectionist, Sensitive]",Yes,Yes,Yes,Pass,Data
0,Michel Lebarree,07/08/2019,"{'Python': 3, 'Java': 4, 'Ruby': 1, 'R': 2, 'P...",[Versatile],"[Controlling, Perfectionist, Chatty]",Yes,Yes,Yes,Pass,Engineering
...,...,...,...,...,...,...,...,...,...,...
0,Jacky Reilingen,04/04/2019,"{'C#': 2, 'Java': 6, 'R': 1, 'SPSS': 4}",[Versatile],"[Indifferent, Intolerant, Introverted]",Yes,No,Yes,Fail,Business
0,Phillis Lyfield,10/04/2019,"{'C#': 4, 'Java': 4, 'Ruby': 4, 'PHP': 1}","[Organisation, Independent, Determined]","[Sensitive, Overbearing, Impatient]",Yes,Yes,Yes,Pass,Engineering
0,Celle Barlas,16/04/2019,"{'R': 2, 'C++': 1, 'JavaScript': 4}",[Problem Solving],[Critical],Yes,Yes,Yes,Pass,Engineering
0,Scott Duny,11/04/2019,{'Ruby': 3},"[Reliable, Perfectionism, Problem Solving]",[Stubborn],Yes,Yes,Yes,Pass,Data


In [172]:
df.dtypes

name                      object
date                      object
tech_self_score           object
strengths                 object
weaknesses                object
self_development          object
geo_flex                  object
financial_support_self    object
result                    object
course_interest           object
dtype: object

In [173]:
new_df = df

# Filling Null values with a string 'N/A'
new_df = new_df.fillna('N/A')

# Converting columns values to strings and capitalizing them
columns_to_convert = ['name', 'self_development', 'geo_flex', 'financial_support_self', 'result', 'course_interest']
new_df[columns_to_convert] = new_df[columns_to_convert].astype(str).apply(lambda x: x.str.upper())

# Converting 'self_development', 'geo_flex', 'financial_support_self', 'result' to boolean values
mapping = {'Yes': True, 'No': False, 'Pass': True, 'Fail': False}
new_df[['self_development', 'geo_flex', 'financial_support_self', 'result']] = new_df[['self_development', 'geo_flex', 'financial_support_self', 'result']].replace(mapping)

# Defininf a function to check for dictionaries and change the data types inside
def update_dict_format(df_dict):
    if isinstance(df_dict, dict):
        # Converting all keys to capitalised strings and values to integers
        return {str(k).upper(): int(v) for k, v in df_dict.items()}
    return df_dict

# Applying a fucntion to dictionaries
new_df = new_df.applymap(update_dict_format)

new_df['date'] = pd.to_datetime(new_df['date'], dayfirst=True)
new_df['date'] = new_df['date'].str.replace('//', '/')




In [174]:
new_df['tech_self_score']

0        {'C#': 6, 'JAVA': 5, 'R': 2, 'JAVASCRIPT': 2}
0          {'PYTHON': 1, 'C#': 4, 'JAVA': 2, 'C++': 4}
0                                {'RUBY': 4, 'C++': 4}
0                               {'JAVA': 3, 'SPSS': 4}
0    {'PYTHON': 3, 'JAVA': 4, 'RUBY': 1, 'R': 2, 'P...
                           ...                        
0              {'C#': 2, 'JAVA': 6, 'R': 1, 'SPSS': 4}
0            {'C#': 4, 'JAVA': 4, 'RUBY': 4, 'PHP': 1}
0                  {'R': 2, 'C++': 1, 'JAVASCRIPT': 4}
0                                          {'RUBY': 3}
0          {'PYTHON': 2, 'C#': 3, 'JAVA': 3, 'C++': 4}
Name: tech_self_score, Length: 3105, dtype: object